In [1]:
# create a hdf5 dataset out of goetiff files

In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

from glob import glob
#from os import listdir
#from os.path import splitext
import logging
import os

In [2]:
# directory paths
#dir_img = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/data_256_ortho/'
#dir_mask = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/data_256_gt_roads/'
dir_img = '/home/philipp/Data/edin_dataset/'

In [3]:
sufix = 'ortho/'
ids_ortho = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'dsm/'
ids_dsm = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'dtm/'
ids_dtm = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'slope/'
ids_slope = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]

In [4]:
sufix = 'ortho'
ids_ortho = []
for i in range(100):
    ids_ortho.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')
    
sufix = 'dsm'
ids_dsm = []
for i in range(100):
    ids_dsm.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')
    
sufix = 'dtm'
ids_dtm = []
for i in range(100):
    ids_dtm.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')
    
sufix = 'slope'
ids_slope = []
for i in range(100):
    ids_slope.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')

In [5]:
print(ids_ortho[:4])
print(ids_dsm[:4])
print(ids_dtm[:4])
print(ids_slope[:4])

['/home/philipp/Data/edin_dataset/ortho/tile_ortho_0.tif', '/home/philipp/Data/edin_dataset/ortho/tile_ortho_1.tif', '/home/philipp/Data/edin_dataset/ortho/tile_ortho_2.tif', '/home/philipp/Data/edin_dataset/ortho/tile_ortho_3.tif']
['/home/philipp/Data/edin_dataset/dsm/tile_dsm_0.tif', '/home/philipp/Data/edin_dataset/dsm/tile_dsm_1.tif', '/home/philipp/Data/edin_dataset/dsm/tile_dsm_2.tif', '/home/philipp/Data/edin_dataset/dsm/tile_dsm_3.tif']
['/home/philipp/Data/edin_dataset/dtm/tile_dtm_0.tif', '/home/philipp/Data/edin_dataset/dtm/tile_dtm_1.tif', '/home/philipp/Data/edin_dataset/dtm/tile_dtm_2.tif', '/home/philipp/Data/edin_dataset/dtm/tile_dtm_3.tif']
['/home/philipp/Data/edin_dataset/slope/tile_slope_0.tif', '/home/philipp/Data/edin_dataset/slope/tile_slope_1.tif', '/home/philipp/Data/edin_dataset/slope/tile_slope_2.tif', '/home/philipp/Data/edin_dataset/slope/tile_slope_3.tif']


In [36]:
from osgeo import gdalconst

def tif2array(input_file, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    Inputs:
        input_file (str) : the name of input GeoTiff file.
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)
        
    #image = image[2:-2,2:-2,:]
        
    return image

In [7]:
# convert image to numpy array
img = tif2array(ids_ortho[10], np.uint8)
print(ids_ortho[10])

1
515 515 4
/home/philipp/Data/edin_dataset/ortho/tile_ortho_10.tif


In [8]:
img.shape

(515, 515, 4)

In [9]:
img

array([[[ 76,  83,  76, 164],
        [ 78,  87,  79, 167],
        [ 62,  64,  53, 150],
        ...,
        [ 42,  46,  46, 111],
        [ 40,  43,  43, 106],
        [ 40,  41,  42, 106]],

       [[ 70,  76,  68, 155],
        [ 81,  91,  84, 169],
        [ 62,  65,  55, 149],
        ...,
        [ 42,  46,  46, 111],
        [ 44,  46,  48, 115],
        [ 42,  44,  46, 114]],

       [[ 60,  64,  57, 139],
        [ 69,  76,  68, 156],
        [ 61,  64,  55, 145],
        ...,
        [ 41,  44,  45, 108],
        [ 46,  50,  51, 122],
        [ 44,  46,  47, 116]],

       ...,

       [[ 86,  73,  53, 183],
        [ 70,  57,  42, 171],
        [ 86,  77,  56, 190],
        ...,
        [ 82,  81,  58, 187],
        [ 94,  97,  70, 197],
        [108, 115,  89, 209]],

       [[ 95,  86,  61, 192],
        [ 93,  84,  61, 192],
        [ 98,  92,  68, 198],
        ...,
        [ 73,  70,  49, 180],
        [106, 114,  91, 209],
        [114, 126, 105, 216]],

       [[126

In [10]:
img.dtype

dtype('uint8')

In [78]:
# convert image to numpy array
img_ortho = tif2array(ids_ortho[10], np.uint8)
print(ids_ortho[10])
print(img_ortho.shape)
print(img_ortho.dtype)

# convert image to numpy array
img_dsm = tif2array(ids_dsm[15], np.float32)
print(ids_dsm[15])
print(img_dsm.shape)
print(img_dsm.dtype)

# convert image to numpy array
img_dtm = tif2array(ids_dtm[15], np.float32)
print(ids_dtm[15])
print(img_dtm.shape)
print(img_dtm.dtype)

# convert image to numpy array
img_slope = tif2array(ids_slope[15], np.float32)
print(ids_slope[15])
print(img_slope.shape)
print(img_slope.dtype)

/home/philipp/Data/edin_dataset/ortho/tile_ortho_10.tif
(515, 515, 4)
uint8
/home/philipp/Data/edin_dataset/dsm/tile_dsm_15.tif
(516, 516, 1)
float32
/home/philipp/Data/edin_dataset/dtm/tile_dtm_15.tif
(516, 516, 1)
float32
/home/philipp/Data/edin_dataset/slope/tile_slope_15.tif
(516, 516, 1)
float32


In [12]:
img_ortho = img_ortho[1:-2,1:-2]
img_dsm = img_dsm[2:-2,2:-2]
img_dtm = img_dtm[2:-2,2:-2]
img_slope = img_slope[2:-2,2:-2]

In [13]:
print(img_ortho.shape)
print(img_dsm.shape)
print(img_dtm.shape)
print(img_slope.shape)

(512, 512, 4)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)


In [14]:
# Data Standardization
# y = (x - mean) / standard_deviation
# mean = sum(x) / count(x)
# standard_deviation = sqrt( sum( (x - mean)^2 ) / count(x))

In [15]:
# Data Normalization
# y = (x - min) / (max - min)

In [16]:
means = []
stds = []

for img in [img_ortho, img_dsm, img_dtm, img_slope]:
    means.append(np.mean(img, dtype='float32'))
    stds.append(np.std(img, dtype='float32'))
print(means)
print(stds)

[106.623276, 14.312894, 1325.6674, 29.272326]
[54.006954, 12.487458, 16.26564, 7.691262]


In [17]:
nor_img_ortho = (img_ortho - means[0]) / stds[0]
nor_img_dsm = (img_dsm - means[1]) / stds[1]
nor_img_dtm = (img_dtm - means[2]) / stds[2]
nor_img_slope = (img_slope - means[3]) / stds[3]

In [18]:
print(nor_img_ortho.shape)
print(nor_img_ortho.dtype)

(512, 512, 4)
float32


In [20]:
print(nor_img_dsm.shape)
print(nor_img_dsm.dtype)

(512, 512, 1)
float32


In [31]:
np.concatenate([nor_img_ortho, nor_img_dsm, nor_img_dtm, nor_img_slope], axis=2).shape

(512, 512, 7)

In [97]:
def cut_img(img, x, y):
    # set pixel sizes
    x_i, y_i, z_i = img.shape
    # dict to store the sliceing information
    d = {}
    
    for var, var_i, key in [(x, x_i, 'x'), (y, y_i, 'y')]:
        # if image pixel size is grater than the target pixel size
        if (var_i > var):
            # if even cut same amount of pixels from both sides
            if var_i%2 == 0:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub
            # if odd cut 1 pixel more from right/bottom
            else:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub + 1
        else:
            print('image too small')
    # cut image
    img = img[d['x0']:-d['x1'],d['y0']:-d['y1']]
    
    return img

In [147]:
# pass ids_ortho, ids_dsm, ids_dtm, ids_slope
def create_array(ids, dtype):
    
    imgs = []
    
    if dtype == np.uint8:
        # add all
        for i in ids:
            # load image to numpy array
            img = tif2array(i, np.uint8)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            
        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        print(arr.shape)
            
        # calculate mean
        mean = np.mean(arr, dtype='float32')
        # calculate standard deviation
        std = np.std(arr, dtype='float32')
    
    
    
    
    else:
        # add all
        for i in ids:
            
            # load image to numpy array
            img = tif2array(i, np.float32)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            x, y, z = img.shape

        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        arr[arr < 0] = np.nan
        print(arr.shape)

        # calculate mean
        mean = np.nanmean(arr, dtype='float32')
        # calculate standard deviation
        std = np.nanstd(arr, dtype='float32')
        # convert nan to 0
        arr = np.nan_to_num(arr)
    
    # normalize data
    arr = (arr - mean) / std
    
    return arr, mean, std

In [148]:
arr_ortho, mean_ortho, std_ortho = create_array(ids_ortho, np.uint8)
arr_dsm, mean_dsm, std_dsm = create_array(ids_dsm, np.float32)
arr_dtm, mean_dtm, std_dtm = create_array(ids_dtm, np.float32)
arr_slope, mean_slope, std_slope = create_array(ids_slope, np.float32)

(100, 512, 512, 4)
(100, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)


In [150]:
# mean & std
print(mean_ortho, std_ortho)
print(mean_dsm, std_dsm)
print(mean_dtm, std_dtm)
print(mean_slope, std_slope)

# min & max
print(arr_ortho.min(), arr_ortho.max())
print(arr_dsm.min(), arr_dsm.max())
print(arr_dtm.min(), arr_dtm.max())
print(arr_slope.min(), arr_slope.max())

94.71759 48.496964
14.648031 12.43344
1346.2009 139.37895
27.917845 9.733992
-1.6025249 3.3049989
-1.1781157 2.4411561
-1.8267218 2.93939
-2.8608007 4.5668583


In [ ]:
# concatonate

In [155]:
data = []
for i in range(len(arr_ortho)):
    data.append(np.concatenate([arr_ortho[0], arr_dsm[0], arr_dtm[0], arr_slope[0]], axis=2))
# convert list with arrays to numpy array
data = np.stack(data, axis=0)

In [156]:
data.shape

(100, 512, 512, 7)

In [46]:
imgs_ortho = []
# add all 
for ids in ids_ortho:
    # load image to numpy array
    img_ortho = tif2array(ids, np.uint8)
    # cut into right shape
    img_ortho = img_ortho[1:-2,1:-2]
    # append array to list
    imgs_ortho.append(img_ortho)

# convert list with arrays to numpy array
arr_ortho = np.stack(imgs_ortho, axis=0)
print(arr_ortho.shape)

# calculate mean
mean_ortho = np.mean(arr_ortho, dtype='float32')
# calculate standard deviation
std_ortho = np.std(arr_ortho, dtype='float32')

# normalize data
arr_ortho = (arr_ortho - mean_ortho) / std_ortho

(100, 512, 512, 4)


In [47]:
arr_ortho

array([[[[-1.1282684 , -1.0664089 , -1.1076485 , -0.6746317 ],
         [-1.1076485 , -1.0251691 , -1.045789  , -0.6746317 ],
         [-1.1282684 , -1.045789  , -1.0664089 , -0.77773094],
         ...,
         [-1.1695081 , -1.1282684 , -1.1488882 , -0.5715325 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.26223478],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.17975539]],

        [[-1.1488882 , -1.0870287 , -1.1282684 , -0.77773094],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.8395905 ],
         [-1.1488882 , -1.0664089 , -1.1076485 , -0.86021036],
         ...,
         [-1.1076485 , -1.0251691 , -1.0251691 , -0.4890531 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.38595387],
         [-1.0870287 , -1.045789  , -1.045789  , -0.365334  ]],

        [[-1.1488882 , -1.0664089 , -1.1076485 , -0.77773094],
         [-1.1695081 , -1.0664089 , -1.1076485 , -0.8395905 ],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.90145004],
         ...,
         

In [34]:
temp = []
for i in ids:
    temp.append(tif2array(dir_mask + ids[0] + ".tif"))

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

...

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

In [12]:
# hdf5

In [158]:
# initialising hdf5
ds_file_name  = "/home/philipp/Data/edin_dataset/dataset_512.h5"

ortho_shape = (len(arr_ortho), 512, 512, 4)
dsm_shape = (len(arr_dsm), 512, 512, 1)

with h5py.File(ds_file_name, 'w') as hf:
    hf.create_dataset('x_ortho', data=arr_ortho, shape=ortho_shape, dtype=np.float32, chunks=True)
    hf.create_dataset('x_dsm', data=arr_dsm, shape=dsm_shape, dtype=np.float32, chunks=True)
    hf.create_dataset('x_dtm', data=arr_dtm, shape=dsm_shape, dtype=np.float32, chunks=True)
    hf.create_dataset('x_slope', data=arr_slope, shape=dsm_shape, dtype=np.float32, chunks=True)

In [161]:
# read HDF5 file
with h5py.File("/home/philipp/Data/edin_dataset/dataset_512.h5", 'r') as hf:
    dset_x_train = np.array(hf['x_ortho'])
    dset_y_train = np.array(hf['x_dsm'])
    dset_x_test = np.array(hf['x_dtm'])
    dset_y_test = np.array(hf['x_slope'])
    
print(dset_x_train.shape)
print(dset_y_train.shape)
print(dset_x_test.shape)
print(dset_y_test.shape)

(100, 512, 512, 4)
(100, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)


In [162]:
dset_x_train

array([[[[-1.1282684 , -1.0664089 , -1.1076485 , -0.6746317 ],
         [-1.1076485 , -1.0251691 , -1.045789  , -0.6746317 ],
         [-1.1282684 , -1.045789  , -1.0664089 , -0.77773094],
         ...,
         [-1.1695081 , -1.1282684 , -1.1488882 , -0.5715325 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.26223478],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.17975539]],

        [[-1.1488882 , -1.0870287 , -1.1282684 , -0.77773094],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.8395905 ],
         [-1.1488882 , -1.0664089 , -1.1076485 , -0.86021036],
         ...,
         [-1.1076485 , -1.0251691 , -1.0251691 , -0.4890531 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.38595387],
         [-1.0870287 , -1.045789  , -1.045789  , -0.365334  ]],

        [[-1.1488882 , -1.0664089 , -1.1076485 , -0.77773094],
         [-1.1695081 , -1.0664089 , -1.1076485 , -0.8395905 ],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.90145004],
         ...,
         